In [65]:
from oracle import MNIST, OracleMNIST
import numpy
from sklearn.neighbors import KNeighborsClassifier

# dataSource = lambda train: MNIST(root="datasets", train=train, download=True)
dataSource = lambda train: OracleMNIST(root="datasets", train=train, download=True)

trainData: MNIST = dataSource(True)
testData: MNIST = dataSource(False)
trainX = numpy.array(trainData.data).reshape((-1, 28 * 28))
testX = numpy.array(testData.data).reshape((-1, 28 * 28))
print("trainX", trainX.shape, 'testX', testX.shape, "classes", len(trainData.classes))

trainX (60000, 784) testX (10000, 784) classes 10


In [ ]:
import math

# 尝试编写的一个小批样本KNN参数搜索。
# 不过效率与全搜索差不多多少。平均值也难以收敛。
selectTest = 100
bestScore = 0
bestK = 10
scores = [[] for _ in range(100)]
bestTime = 0
while bestTime < 50:
    for k in range(bestK - 3, bestK + 4):
        if k <= 0: continue
        model = KNeighborsClassifier(n_neighbors=k, weights='distance')
        model.fit(trainX, trainData.targets.numpy())
        testI = numpy.random.choice(range(testX.shape[0]), selectTest)
        score = model.score(testX[testI], testData.targets.numpy()[testI])
        scores[k].append(score)
    newBest = numpy.argmax([(numpy.average(x) if len(x) > 0 else 0) for x in scores])
    if newBest != bestK:
        bestTime = 0
        bestK = newBest
    else:
        bestTime += 1
    if bestTime % 10 == 0:
        scoreV = scores[bestK]
        print("BestK=", bestK, "score=%.4f±%.4f" %
              (numpy.average(scoreV), numpy.std(scoreV) / math.sqrt(len(scoreV)) if len(scoreV) > 1 else 1))

BestK= 9 score=0.9733±0.0027
BestK= 6 score=1.0000±1.0000


In [68]:
model = KNeighborsClassifier(n_neighbors=4, weights='distance')
model.fit(trainX, trainData.targets.numpy())
print("BestK=", bestK, "score=", model.score(testX, testData.targets.numpy()))

BestK= 12 score= 0.9714


In [ ]:
# Result: MNIST 0.9714 (k=4,distance)   Oracle 0.64433(k=11,distance)